In [2]:
from __future__ import division

import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import sys

from svgpathtools import parse_path

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

### Setup

In [60]:
# directory & file hierarchy
proj_dir = os.path.abspath('../')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'analysis'))

if not os.path.exists(results_dir):
    os.makedirs(results_dir)  

if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
# Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

<module 'analysis_helpers' from 'analysis_helpers.pyc'>

#### set up connection to mongo

In [107]:
#### set vars 
import pandas as pd
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['svgAnnotation']
coll = db['examples']

# which iteration name should we use?
iterationName = 'testing8'

### get some descriptive stats on our annotation data!

In [108]:
num_annotations = coll.find({'iterationName':iterationName}).count()
print 'We have {} annotations so far.'.format(num_annotations)

We have 2 annotations so far.


#### build up annotation data csv

In [109]:
## TODO remember to ignore researcher worker ID's in our dataset
jefan = ['A1MMCS8S8CTWKU','A1MMCS8S8CTWKV','A1MMCS8S8CTWKS']
hawkrobe = ['A1BOIDKD33QSDK']
megsano = ['A1DVQQLVZR7W6I']
kmukherjee = ['A1WU4IHJNQGVAY']
researchers = jefan + hawkrobe + megsano + kmukherjee

In [110]:
## TODO use a helper function to get complete and valid HITs 

In [111]:
## categories that are in play
main_category = 'chair'

In [112]:
### get all the sketch recs!
sketch_recs = coll.find({'iterationName':iterationName}).sort('time')

### initialize a bunch of stuff
orig_gameID = [] # the gameID from which this sketch was sourced
category = [] # e.g., "chair"
target = [] # e.g., "inlay"
condition = [] # e.g., "closer" vs. "further" or "repeated" vs. "control
trial_num = []
time_submitted = [] # when the participant clicked "next sketch"
time_labeled = [] # unique to each spline labeled
time_clicked = [] # when this spline was clicked/selected
num_strokes_in_sketch = [] # how many strokes in this sketch
num_splines_in_sketch = [] # how many spline elements in this sketch
stroke_num = [] # which stroke number this labeled spline came from
cumulative_spline_num = [] # spline index in the cumulative spline sequence for the entire sketch
within_stroke_spline_num = [] # spline index for the current stroke
label = [] # the label provided by the participant
spline_svg_string = [] # the svg spline string that earned this label
sketch_svg_string = [] # the entire svg string correponding to this sketch

for sketch_ind,sketch in enumerate(sketch_recs):
    print 'Analyzing {} of {} sketches ...'.format(sketch_ind,sketch_recs.count())
    ## get annotations embedded within record
    annotations_string = sketch['annotations']  
    ## convert to json dictionary
    _annotations_dict = json.loads(annotations_string)   
    annotations_dict = _annotations_dict[0][main_category]
    num_splines = len(annotations_dict)
    for annotation in annotations_dict:
        ## get spline-level metadata
        label.append(annotation['label'])
        stroke_num.append(annotation['strokeNum'])
        spline_svg_string.append(annotation['svgString'])
        cumulative_spline_num.append(annotation['cumulativeSplineNum'])
        within_stroke_spline_num.append(annotation['withinStrokeSplineNum'])
        time_clicked.append(annotation['timeClicked'])
        time_labeled.append(annotation['timeLabeled'])
        ## get sketch-level metadata
        orig_gameID.append(sketch['originalGameID'])        
        category.append(sketch['category'])
        target.append(sketch['target'])
        condition.append(sketch['condition'])
        time_submitted.append(sketch['time'])
        trial_num.append(sketch['trialNum'])
        assert sketch['numSplines']==num_splines
        num_splines_in_sketch.append(sketch['numSplines'])
        num_strokes_in_sketch.append(sketch['numStrokes'])
        sketch_svg_string.append(sketch['svg'])
                                

Analyzing 0 of 2 sketches ...


AssertionError: 

In [113]:
sketch['numSplines']

22

In [114]:
len(annotations_dict)

23

In [118]:
roop = []
for doop in annotations_dict:
    roop.append(doop['cumulativeSplineNum'])

In [127]:
len(roop)

23

In [126]:
sorted(roop)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 16,
 17,
 18,
 19,
 20,
 21]

In [122]:
len(np.unique(roop))

22

In [123]:
annotations_dict

[{u'cumulativeSplineNum': 11,
  u'label': u'leg',
  u'strokeColor': [u'Color', 0.57647, 0.52157, 0.48235],
  u'strokeNum': 1,
  u'svgString': u'M139,212c-3.49268,6.98536 -6.75552,13.51104 -10,20c-0.91676,1.83351 -1.65516,8 -3,8',
  u'timeClicked': 1533351094145,
  u'timeLabeled': 1533351095485,
  u'withinStrokeSplineNum': 2},
 {u'cumulativeSplineNum': 20,
  u'label': u'leg',
  u'strokeColor': [u'Color', 0.57647, 0.52157, 0.48235],
  u'strokeNum': 4,
  u'svgString': u'M61,108c-2.0648,6.19439 2.56415,6.69245 4,11c1.05409,3.16228 0,6.66667 0,10c0,29.75973 3,62.46388 3,90',
  u'timeClicked': 1533351094145,
  u'timeLabeled': 1533351095486,
  u'withinStrokeSplineNum': 1},
 {u'cumulativeSplineNum': 13,
  u'label': u'leg',
  u'strokeColor': [u'Color', 0.57647, 0.52157, 0.48235],
  u'strokeNum': 2,
  u'svgString': u'M213,189c0,7.9763 -1,14.83683 -1,23',
  u'timeClicked': 1533351094145,
  u'timeLabeled': 1533351095487,
  u'withinStrokeSplineNum': 1},
 {u'cumulativeSplineNum': 14,
  u'label': u'l

In [82]:
sketch = sketch_recs[0]
## get annotations embedded within record
annotations_string = sketch['annotations']  
## convert to json dictionary
_annotations_dict = json.loads(annotations_string)   
annotations_dict = _annotations_dict[0][main_category]
num_splines = len(annotations_dict)
# for annotation in annotations_dict:

In [88]:
annotations_dict[30]

{u'cumulativeSplineNum': 39,
 u'label': u'backrest',
 u'strokeColor': [u'Color', 0.92941, 0.21961, 0.03137],
 u'strokeNum': 19,
 u'svgString': u'M79,105c1.66667,0 3.33333,0 5,0',
 u'timeClicked': 1533350332615,
 u'timeLabeled': 1533350334651,
 u'withinStrokeSplineNum': 0}

In [90]:
sketch['numSplines']

41